## 講習1 --- IRAFを使ってみる

IRAFのCLコマンドラインを使っての対話的なデータ処理を、 
python + Jupyter notebookからも行うことができます。

pythonからIRAFを使うにはpyrafを利用します。  
pyrafを(便利に)使うためには、  

- ホームディレクトリにirafというディレクトリを作成
- そのディレクトリ内で mkirafを実行し、login.cl を作成
- 必要に応じてそのlogin.clを編集  

をしておきます。 こうしておくと、login.clでの設定がpyraf利用時に反映されます。また、~/iraf/uparm/に各タスク(imexam, imstatなど)のパラメータが保存されます。　　

### モジュールの読み込み

In [1]:
from pyraf import iraf 

これでirafのタスクをpythonで関数として使うことができます。('Warning : sscanf library not ... 'がでることがあります。とりえあず無視してください。)

Unixコマンドを使い、カレントディレクトリおよびその中身を確認することができます。

In [2]:
pwd

'/Users/yas.nakajima/ads2017python'

In [3]:
ls

010604.ffiles/             course2_numpy.ipynb
ap.coo                     course3_pyfits.ipynb
ap.mag                     course4_matplotlib.ipynb
aplpy/                     course5_IRAF_apphot.ipynb
appendix.ipynb             imstat.ipynb
appendix1_aplpy.ipynb      my10x10.fits
coo.txt                    myout.fits
course0_module.ipynb       myout2.fits
course1_IRAF_1.ipynb       sample_data/


ffilesのディレクトリの中にサンプルの画像FITSファイルがあります。IRAFのコマンド、display, imexam, imstatを使って、それらファイルの表示やデータの吟味、統計量の測定をしてみましょう。

### DS9にFITSデータを表示してみる

別のターミナルからds9を立ち上げておきます。

In [4]:
iraf.display('./010604.ffiles/hf0001.fits', 1)

z1=977.7808 z2=1139.195


In [5]:
iraf.display('./010604.ffiles/hf0002.fits', 2)

z1=971.7075 z2=1131.667


表示されましたね。  
irafモジュールのdisplay()関数を使い、引数としてファイルパス、ds9のフレーム番号、などを指定します。

### imexamも使ってみよう
ds9にFITS画像を表示した状態で、次のコマンドを実行し、マウスカーソルを、星のないところにあてて**m**、あるいは星にあてて**a**をタイプしてみましょう。終わるときには**q**をタイプしてください。

In [6]:
iraf.imexam()

imexamでは、**r**とか**e**コマンドを使うとさらに別のグラフィックウィンドウが現れます。ブラウザとかの後ろに隠れているかもしれません。

In [7]:
iraf.imexam('./010604.ffiles/hf0001.fits', 1) 

#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
   [325:329,615:619]       25    1052.    1057.    16.88    1016.    1082.
   [638:642,646:650]       25    1060.    1059.    17.57    1023.    1100.
#   COL    LINE   COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
 499.00  489.85 499.00 489.85
   5.57  13.47  41064.   1058.   7268. 0.11   45 4.34     1.87     2.04   1.86
 463.69  478.31 463.69 478.31
   6.28  12.99  63916.   1058.  10434. 0.08   39 11.2     1.98     2.23   2.09


### imstat も使ってみる
FITS画像の統計量を調べるタスクimstatを使ってみましょう。

In [8]:
iraf.imstat.unlearn()  #  パラメータをデフォルト値に

In [9]:
iraf.imstat('./010604.ffiles/hf0001.fits')

#               IMAGE      NPIX      MEAN    STDDEV       MIN       MAX
 ./010604.ffiles/hf0001.fits   1048576     893.7     4103.  -199198.    25331.


### eparでパラメータ設定 
**iraf.epar('タスク名')** で、IRAFでおなじみのeparのパラメータ設定の画面が別ウィンドウで開きます。GUIでパラメータ設定ができます。同じGUIのウインドウでHELPも読めます。   
(2017-05 MacOS Sierra + Python3.5.3 + Jupyter Notebook では iraf.epar('display')でUnicodeDecodeErrorが生じる)

In [10]:
iraf.epar('imstat')

### 変数としてパラメータ設定

Cellでタスクの変数にパラメータを代入してやります。この方法だと、使ったパラメータがこのノートブックに残るので、あとになって「この処理でどんなパラメータ使ったっけ？」となったときに助けになります。

In [11]:
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.lower = 0
iraf.imstat.upper = 10000

In [12]:
iraf.imstat('./010604.ffiles/hf0001.fits')

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13


In [13]:
print (iraf.imstat.fields)

midpt, mean, stddev


In [14]:
iraf.imstat.images = './010604.ffiles/hf0001.fits'

In [15]:
iraf.imstat()

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13


上ではimstat()の結果が標準出力に表示されました。それぞれの値を変数に保存するには次のようにします。

In [16]:
out = iraf.imstat(format='no', Stdout=1) # format='no' でヘッダ行非表示、Stdout=1で戻り値を返す
v = out[0].split()  #  戻り値はリスト
median = float(v[0])   # 文字列をfloatに変換しておく
mean = float(v[1])
stddev = float(v[2])
print (median)
print (mean)
print ('{0:.2f}'.format(stddev))   #  フォーマット文を使ってみた

1059.173
1059.59
56.13


デフォルトではStdout=0なので標準出力に値が返ります。

### helpドキュメント

このノートブック内でhelpを読むこともできます。  
helpの表示が縦に長すぎる場合、左の余白部分(In[]: の下あたり)をクリックするとスクロールバーつきウインドウ表示になります。
(私の環境でブラウザChromeの場合にはデフォルトでスクロールバー表示されます。)

In [17]:
#iraf.help('imstat')   #  Githubでは表示が長くなってしまうのでコメントアウトしておきます

### 一括処理

簡単なプログラムで'./010604.ffiles'ディレクトリ内のh-bandのfitsファイル全てについてimstatしましょう。

In [18]:
import glob # pythonの組み込みモジュール。ワイルドカードを使ったファイル処理など。

In [19]:
fitslist = glob.glob('./010604.ffiles/h*fits')

In [20]:
print (fitslist)

['./010604.ffiles/hf0001.fits', './010604.ffiles/hf0002.fits', './010604.ffiles/hf0003.fits', './010604.ffiles/hf0004.fits', './010604.ffiles/hf0005.fits']


In [21]:
for img in fitslist:
    iraf.imstat(img)

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13
#     MIDPT      MEAN    STDDEV
      1052.     1053.     52.06
#     MIDPT      MEAN    STDDEV
      1043.     1043.     58.26
#     MIDPT      MEAN    STDDEV
      1018.     1020.     46.33
#     MIDPT      MEAN    STDDEV
      1011.     1012.     42.15


上では、glob.globで抽出したリストをいったん変数に代入しましたが、以下のようにしても十分です。

In [22]:
for img in glob.glob('./010604.ffiles/h*fits'):
    iraf.imstat(img)

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13
#     MIDPT      MEAN    STDDEV
      1052.     1053.     52.06
#     MIDPT      MEAN    STDDEV
      1043.     1043.     58.26
#     MIDPT      MEAN    STDDEV
      1018.     1020.     46.33
#     MIDPT      MEAN    STDDEV
      1011.     1012.     42.15


### 練習問題 
未定  